# Языковые модели на n-граммах

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://www.nltk.org/api/nltk.util.html
* https://web.stanford.edu/~jurafsky/slp3/3.pdf
* https://www.youtube.com/watch?v=QGT6XTeA3YQ

## Задачи для совместного разбора

1\. Выделите из текста n-граммы.

In [ ]:
text = """Вода это жидкость которая имеет свойство быть водой.
Вода состоит из молекул, которые выглядят как вода"""

In [ ]:
from nltk.util import ngrams
from nltk import sent_tokenize
from nltk.tokenize import RegexpTokenizer

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# unigrams = ngrams(text, n=1)
# unigrams = list(unigrams)
# unigrams

In [ ]:
sents = sent_tokenize(text)
tokenizer = RegexpTokenizer("\w+")

unigrams = []
bigrams = []
for sent in sents:
  tokens = tokenizer.tokenize(sent.lower())
  unigrams.extend(ngrams(tokens, n=1))
  bigrams.extend(ngrams(tokens, n=2))

In [ ]:
unigrams[:5]

[('вода',), ('это',), ('жидкость',), ('которая',), ('имеет',)]

In [ ]:
bigrams[:5]

[('вода', 'это'),
 ('это', 'жидкость'),
 ('жидкость', 'которая'),
 ('которая', 'имеет'),
 ('имеет', 'свойство')]

2. Рассчитайте вероятности  $P(вода)$, $P(это|вода)$, $P(состоит|вода)$.

$$P(w_i) = \frac{C(w_i)}{N}$$
$$P(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)}{C(w_{i-1})}$$

In [ ]:
w = ("вода", )
c = unigrams.count(w)
N = len(unigrams)
c, N, c / N

(3, 16, 0.1875)

In [ ]:
# P(это|вода)
w2 = ("вода", "это")
c2 = bigrams.count(w2)
c2, c, c2/c

# P(это)

(1, 3, 0.3333333333333333)

3. Рассчитайте вероятности  $P_L(вода)$, $P_L(это|вода)$, $P_L(состоит|вода)$.

$$P_L(w_i) = \frac{C(w_i)+1}{N+V}$$, $$P_L(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)+1}{C(w_{i-1})+V}$$

In [ ]:
len(set(unigrams))

14

In [ ]:
# вода не
# P(не|вода) = 0
# P  = (0 + 1) / (3 + 14)

## Задачи для самостоятельного решения

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [84]:
import pandas as pd
from nltk.util import ngrams
from nltk import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
import re
import numpy as np
import nltk
import random
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

<p class="task" id="1"></p>

1\. Считайте файл `data/moya-semia/Лучше кошки зверя нет 2.csv`. Получите список предложений из сообщений. Приведите предложения к нижнему регистру и удалите все символы, кроме букв и пробелов. Получите список слов (униграмм) и биграмм.

In [5]:
df = pd.read_csv('/content/drive/MyDrive/NLP/data/moya-semia/Лучше кошки зверя нет 2.csv', header = None)
df.head()

,0,1,2
0,http://forum.moya-semya.ru/index.php?app=forum...,Завтра мои котейки идут к врачу. А 21-го жду и...,Unknown
1,http://forum.moya-semya.ru/index.php?app=forum...,Котя скучает по мужу. Со вторника спит на его ...,Unknown
2,http://forum.moya-semya.ru/index.php?app=forum...,"Получилось так, что мне пришлось ""отдать"" мою ...",Unknown
3,http://forum.moya-semya.ru/index.php?app=forum...,Лучше всего для начала спросить об этом у люде...,Unknown
4,http://forum.moya-semya.ru/index.php?app=forum...,"что волонтеры принесут, то и насыпалиЗначит, к...",Unknown


In [40]:
text = ' '.join(df[1].to_list())
sents = sent_tokenize(text)
sents = [re.sub(r'[^a-zA-Zа-яА-Я\s]', '', sent).lower() for sent in sents]
random.sample(sents, 5)

['точнее  рассказываем и\xa0 делимся их хулиганствами',
 'если москитную сетку не смените на антикошачью будете кошку гагариным зватьпридурками я их зову',
 'пришлось шуршать',
 'но вот в лицо начала кидаться или на шею изза угла',
 'не я конечно с хомячихой']

In [38]:
unigrams = []
bigrams = []
for sent in sents:
    words = word_tokenize(sent)
    unigrams.extend(ngrams(words, n=1))
    bigrams.extend(ngrams(words, n=2))

In [42]:
random.sample(unigrams, 5)

[('следующий',), ('жалобно',), ('смыласьвот',), ('кусок',), ('приволок',)]

In [47]:
random.sample(bigrams, 5)

[('что', 'не'),
 ('ухо', 'востро'),
 ('гнал', 'с'),
 ('человеческой', 'едойможет'),
 ('тарелке', 'с')]

<p class="task" id="2"></p>

2\. Получите распределение вероятностей для униграм $P(w_i) = \frac{C(w_i)}{N}$, где $N$ - количество униграм, $C(w_i)$ - частота использования токена $w_i$. Получите распределение условных вероятностей для биграмм $P(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)}{C(w_{i-1})}$ ($C(w_{i-1} w_i)$ - частота использования словосочетания $w_{i-1}w_i$).

In [49]:
unigrams_set = set(unigrams)
N = len(unigrams)
unigrams_distribution = {unigram : unigrams.count(unigram) / N for unigram in unigrams_set}
unigrams_distribution

{('изучил',): 1.1789533252378538e-05,
 ('гребет',): 1.1789533252378538e-05,
 ('нормальнойоня',): 5.894766626189269e-06,
 ('устроила',): 3.5368599757135615e-05,
 ('стакан',): 3.5368599757135615e-05,
 ('требовался',): 5.894766626189269e-06,
 ('закрытьтакже',): 5.894766626189269e-06,
 ('порхал',): 5.894766626189269e-06,
 ('восстановлено',): 5.894766626189269e-06,
 ('зацепил',): 5.894766626189269e-06,
 ('разгребал',): 5.894766626189269e-06,
 ('чуя',): 5.894766626189269e-06,
 ('диагонали',): 5.894766626189269e-06,
 ('приборами',): 5.894766626189269e-06,
 ('переживает',): 5.894766626189269e-05,
 ('приподнял',): 1.1789533252378538e-05,
 ('легкости',): 5.894766626189269e-06,
 ('улягутся',): 5.894766626189269e-06,
 ('разделкой',): 5.894766626189269e-06,
 ('прокормлю',): 5.894766626189269e-06,
 ('игрыа',): 5.894766626189269e-06,
 ('отморожу',): 5.894766626189269e-06,
 ('соревнование',): 5.894766626189269e-06,
 ('насыпаешь',): 5.894766626189269e-06,
 ('вслед',): 4.1263366383324885e-05,
 ('поднесл

In [56]:
bigrams_set = set(bigrams)
bigrams_distribution = {bigram : bigrams.count(bigram) / unigrams.count((bigram[0], )) for bigram in bigrams_set}
bigrams_distribution

{('входных', 'дверей'): 0.6666666666666666,
 ('не', 'разбегаясь'): 0.00045045045045045046,
 ('что', 'дом'): 0.00047214353163361664,
 ('и', 'выключить'): 0.00015735641227380016,
 ('рискнула', 'ехать'): 0.25,
 ('балконной', 'двери'): 1.0,
 ('и', 'жадно'): 0.00015735641227380016,
 ('позвонила', 'два'): 0.14285714285714285,
 ('говорили', 'чтобы'): 0.25,
 ('эта', 'с'): 0.013157894736842105,
 ('смотреть', 'буду'): 0.020833333333333332,
 ('шикарная', 'такая'): 0.4,
 ('обед', 'по'): 0.13333333333333333,
 ('же', 'хорошая'): 0.001941747572815534,
 ('ещ', 'по'): 0.01020408163265306,
 ('град', 'оплеух'): 1.0,
 ('она', 'уж'): 0.0011337868480725624,
 ('охреначил', 'меня'): 1.0,
 ('через', 'штакетник'): 0.005747126436781609,
 ('не', 'знаешь'): 0.0006756756756756757,
 ('висела', 'на'): 0.42857142857142855,
 ('есть', 'еду'): 0.0028169014084507044,
 ('тося', 'моя'): 0.3333333333333333,
 ('общем', 'дело'): 0.036585365853658534,
 ('специально', 'нет'): 0.06666666666666667,
 ('обработка', 'повторились'): 0

<p class="task" id="3"></p>

3\.Воспользовавшись полученными вероятностями, сгенерируйте текст длиной не более 20 символов, начинающийся с токена "мой". При генерации текста выбирайте слово с наибольшей вероятностью соответствующего биграмма. Выведите полученный текст на экран.

In [77]:
sorted_bigrams_distribution = {k: v for k, v in sorted(bigrams_distribution.items(), key=lambda item: item[1], reverse = True)}

word = 'мой'

generated_text = [word]

for i in range(20):
    for j in sorted_bigrams_distribution.keys():
        if j[0] == word:
            generated_text.append(j[1])
            word = j[1]
            break

print(*generated_text)

мой кот в доме не знаю что я не знаю что я не знаю что я не знаю что я не


<p class="task" id="4"></p>

4\.Воспользовавшись полученными вероятностями, сгенерируйте текст длиной не более 20 символов, начинающийся с токена "мой". При генерации текста выбирайте слово пропорционально вероятностям соответствующих биграммов. Выведите полученный текст на экран.

<p class="task" id="5"></p>

5\. Получите распределение вероятностей для униграм, воспользовавшись сглаживанием Лапласа: $P_L(w_i) = \frac{C(w_i)+1}{N+V}$, где $V$ - количество уникальных униграмм. Получите распределение условных вероятностей для биграмм $P_L(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)+1}{C(w_{i-1})+V}$

In [79]:
unigrams_set = set(unigrams)
V = len(unigrams_set)
N = len(unigrams)
unigrams_distribution_laplas = {unigram : (unigrams.count(unigram) + 1) / (N + V) for unigram in unigrams_set}
unigrams_distribution_laplas

{('изучил',): 1.4923220032930572e-05,
 ('гребет',): 1.4923220032930572e-05,
 ('нормальнойоня',): 9.948813355287047e-06,
 ('устроила',): 3.482084674350467e-05,
 ('стакан',): 3.482084674350467e-05,
 ('требовался',): 9.948813355287047e-06,
 ('закрытьтакже',): 9.948813355287047e-06,
 ('порхал',): 9.948813355287047e-06,
 ('восстановлено',): 9.948813355287047e-06,
 ('зацепил',): 9.948813355287047e-06,
 ('разгребал',): 9.948813355287047e-06,
 ('чуя',): 9.948813355287047e-06,
 ('диагонали',): 9.948813355287047e-06,
 ('приборами',): 9.948813355287047e-06,
 ('переживает',): 5.4718473454078765e-05,
 ('приподнял',): 1.4923220032930572e-05,
 ('легкости',): 9.948813355287047e-06,
 ('улягутся',): 9.948813355287047e-06,
 ('разделкой',): 9.948813355287047e-06,
 ('прокормлю',): 9.948813355287047e-06,
 ('игрыа',): 9.948813355287047e-06,
 ('отморожу',): 9.948813355287047e-06,
 ('соревнование',): 9.948813355287047e-06,
 ('насыпаешь',): 9.948813355287047e-06,
 ('вслед',): 3.979525342114819e-05,
 ('поднесла'

In [80]:
bigrams_set = set(bigrams)
V = len(bigrams_set)
bigrams_distribution_laplas = {bigram : (bigrams.count(bigram) + 1) / (unigrams.count((bigram[0], )) + V) for bigram in bigrams_set}
bigrams_distribution_laplas

{('входных', 'дверей'): 2.8756014799762284e-05,
 ('не', 'разбегаясь'): 2.758290962919375e-05,
 ('что', 'дом'): 1.8789752069221446e-05,
 ('и', 'выключить'): 1.8070438569544083e-05,
 ('рискнула', 'ехать'): 1.9170492777516845e-05,
 ('балконной', 'двери'): 1.917104405505924e-05,
 ('и', 'жадно'): 1.8070438569544083e-05,
 ('позвонила', 'два'): 1.916994153167833e-05,
 ('говорили', 'чтобы'): 1.9170492777516845e-05,
 ('эта', 'с'): 1.9157271621375683e-05,
 ('смотреть', 'буду'): 1.916241101455385e-05,
 ('шикарная', 'такая'): 2.8755463538072233e-05,
 ('обед', 'по'): 2.8752707546627308e-05,
 ('же', 'хорошая'): 1.907705221389191e-05,
 ('ещ', 'по'): 2.8702915259426517e-05,
 ('град', 'оплеух'): 1.917104405505924e-05,
 ('она', 'уж'): 1.901050330307495e-05,
 ('охреначил', 'меня'): 1.917104405505924e-05,
 ('через', 'штакетник'): 1.9139305434605777e-05,
 ('не', 'знаешь'): 3.6777212838925e-05,
 ('висела', 'на'): 3.833988306335666e-05,
 ('есть', 'еду'): 1.9106211429335678e-05,
 ('тося', 'моя'): 1.9170676533

<p class="task" id="6"></p>

6\.Воспользовавшись полученными после сглаживания вероятностями, сгенерируйте текст длиной не более 20 символов, начинающийся с токена "мой". При генерации текста выбирайте слово пропорционально вероятностям соответствующих биграммов. Выведите полученный текст на экран.

<p class="task" id="7"></p>

7\. Рассчитайте перплексию для текста "Котя пришел домой с хромой лапой" для четырех моделей: на 1/2-граммах и с/без использования сглаживания Лапласа. Сведите результат в таблицу. Повторите вычисления для текста "После пар я поеду кормить своего кота", используя доступные модели.

$Perplexity(W) = P(w_1w_2...w_N)^{-\frac{1}{N}}$

Для модели на униграммах $P(w_1w_2...w_N) = \Pi_{i=1}^{N}{P(w_i)}$

Для модели на биграммах $P(w_1w_2...w_N) = \Pi_{i=1}^{N}{P(w_i|w_{i-1})}$

In [81]:
test1 = "После пар я поеду кормить своего кота".lower()
test2 = "Котя пришел домой с хромой лапой".lower()

### test 1

In [149]:
unigrams = []
bigrams = []

words = word_tokenize(test1)
unigrams.extend(ngrams(words, n=1))
bigrams.extend(ngrams(words, n=2))

In [150]:
unigrams_counts = np.unique(unigrams, return_counts = True)
unigrams_counts = dict(zip(unigrams_counts[0], unigrams_counts[1]))
unigrams_counts

{'кормить': 1,
 'кота': 1,
 'пар': 1,
 'поеду': 1,
 'после': 1,
 'своего': 1,
 'я': 1}

In [151]:
bigrams_counts = np.unique(bigrams, return_counts = True, axis = 0)
bigrams_counts = {tuple(words): count for words, count in zip(bigrams_counts[0], bigrams_counts[1])}
bigrams_counts

{('кормить', 'своего'): 1,
 ('пар', 'я'): 1,
 ('поеду', 'кормить'): 1,
 ('после', 'пар'): 1,
 ('своего', 'кота'): 1,
 ('я', 'поеду'): 1}

In [152]:
unigrams_perplexity = np.prod(np.array(list(unigrams_counts.values())) / len(unigrams))**(-1 / len(unigrams))
unigrams_perplexity

7.0

In [153]:
product = 1

for i in bigrams_counts:
    unigram_count = unigrams_counts[i[0]]
    bigram_count = bigrams_counts[i]
    product *= bigram_count / unigram_count

bigrams_perplexity = product**(-1 / len(bigrams_counts))
bigrams_perplexity

1.0

In [154]:
unigrams_perplexity_laplas = np.prod((np.array(list(unigrams_counts.values())) + 1) / (len(unigrams_counts) + len(set(unigrams_counts))))**(-1 / len(unigrams))
unigrams_perplexity_laplas

7.0

In [155]:
product = 1

for i in bigrams_counts:
    unigram_count = unigrams_counts[i[0]]
    bigram_count = bigrams_counts[i]
    product *= (bigram_count + 1) / (unigram_count + len(set(bigrams_counts)))

bigrams_perplexity_laplas = product**(-1 / len(bigrams_counts))
bigrams_perplexity_laplas

3.5

### test 2

In [135]:
unigrams = []
bigrams = []

words = word_tokenize(test2)
unigrams.extend(ngrams(words, n=1))
bigrams.extend(ngrams(words, n=2))

In [137]:
unigrams_counts_2 = np.unique(unigrams, return_counts = True)
unigrams_counts_2 = dict(zip(unigrams_counts_2[0], unigrams_counts_2[1]))
unigrams_counts_2

{'домой': 1, 'котя': 1, 'лапой': 1, 'пришел': 1, 'с': 1, 'хромой': 1}

In [138]:
bigrams_counts_2 = np.unique(bigrams, return_counts = True, axis = 0)
bigrams_counts_2 = {tuple(words): count for words, count in zip(bigrams_counts_2[0], bigrams_counts_2[1])}
bigrams_counts_2

{('домой', 'с'): 1,
 ('котя', 'пришел'): 1,
 ('пришел', 'домой'): 1,
 ('с', 'хромой'): 1,
 ('хромой', 'лапой'): 1}

In [139]:
unigrams_perplexity_2 = np.prod(np.array(list(unigrams_counts_2.values())) / len(unigrams))**(-1 / len(unigrams))
unigrams_perplexity_2

6.0

In [140]:
product = 1

for i in bigrams_counts_2:
    unigram_count = unigrams_counts_2[i[0]]
    bigram_count = bigrams_counts_2[i]
    product *= bigram_count / unigram_count

bigrams_perplexity_2 = product**(-1 / len(bigrams_counts_2))
bigrams_perplexity_2

1.0

In [141]:
unigrams_perplexity_laplas_2 = np.prod((np.array(list(unigrams_counts_2.values())) + 1) / (len(unigrams_counts_2) + len(set(unigrams_counts_2))))**(-1 / len(unigrams))
unigrams_perplexity_laplas_2

6.0

In [142]:
product = 1

for i in bigrams_counts_2:
    unigram_count = unigrams_counts_2[i[0]]
    bigram_count = bigrams_counts_2[i]
    product *= (bigram_count + 1) / (unigram_count + len(set(bigrams_counts_2)))

bigrams_perplexity_laplas_2 = product**(-1 / len(bigrams_counts_2))
bigrams_perplexity_laplas_2

3.0000000000000004

In [163]:
index = ['test1', 'test2']
columns = pd.MultiIndex.from_tuples([
    ('unigrams', 'no Laplace'),
    ('unigrams', 'Laplace'),
    ('bigrams', 'no Laplace'),
    ('bigrams', 'Laplace')
])

data = pd.DataFrame(index=index, columns=columns)

data.loc['test1', ('unigrams', 'no Laplace')] = unigrams_perplexity
data.loc['test1', ('unigrams', 'Laplace')] = unigrams_perplexity_laplas
data.loc['test1', ('bigrams', 'no Laplace')] = bigrams_perplexity
data.loc['test1', ('bigrams', 'Laplace')] = bigrams_perplexity_laplas
data.loc['test2', ('unigrams', 'no Laplace')] = unigrams_perplexity_2
data.loc['test2', ('unigrams', 'Laplace')] = unigrams_perplexity_laplas_2
data.loc['test2', ('bigrams', 'no Laplace')] = bigrams_perplexity_2
data.loc['test2', ('bigrams', 'Laplace')] = bigrams_perplexity_laplas_2
data

unigrams            bigrams        
      no Laplace Laplace no Laplace Laplace
test1        7.0     7.0        1.0     3.5
test2        6.0     6.0        1.0     3.0